In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import matplotlib as mpl
from tqdm import tqdm


from holodeck import plot, detstats
import holodeck.anisotropy as anis
from holodeck.constants import YR, MSOL, MPC
import holodeck as holo
import healpy as hp

import hasasia.sim as hsim
import os


In [ ]:
SHAPE = None
NREALS = 500
# NREALS = 20
NFREQS = 40
NLOUDEST = 10
BGL = 1

BUILD_ARRAYS = False
SAVEFIG = True
TOL=0.01
MAXBADS=5

NVARS = 21
# NVARS = 6

NPSRS = 40
NSKIES = 100
# NSKIES = 15

SHOW_GW=True
ALL_TARGETS = ['hard_time', 'gsmf_phi0', 'gsmf_mchar0_log10', 'mmb_mamp_log10', 'mmb_scatter_dex', 'hard_gamma_inner']

LMAX = 8
NSIDE = 8

In [ ]:
cmap_Blues = plot.truncate_colormap('Blues', 0.4, 1)
cmap_Greens = plot.truncate_colormap('Greens', 0.4, 1)
cmap_Oranges = plot.truncate_colormap('Oranges', 0.4, 1)


In [ ]:
def draw_anis_vars(ax, target, xx, yy_cl, params_cl, colors, parvars):
    ll=1
    handles = []
    for vv, var in enumerate(parvars):
        yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
        pp = 68
        med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        hh, = ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv])
        handles.append(hh)
        ax.fill_between(xx, *conf, alpha=0.25, color=colors[vv])
    return handles
    

In [ ]:
BUILD_ARRAYS=False

xlabel = plot.LABEL_GW_FREQUENCY_YR
ylabel = r"$C_1/C_0$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
cmaps = [
    cmap_Greens, cmap_Greens,
    cmap_Oranges, cmap_Oranges,
    cmap_Blues, cmap_Blues
]
parvars = [0,10,20]

fig, axs = plot.figax_single(ncols=2, nrows=3, height=7,
                             sharey=True, sharex=True)
fig.text(0.55, 0.08, xlabel, va='center', ha='center')
fig.text(0.05, 0.55, ylabel, va='center', ha='center', rotation='vertical')

for ii, ax in enumerate(axs.flatten()):
    target = targets[ii]
    if BUILD_ARRAYS:
        yy_cl, xx_fobs, params_cl = detstats.build_anis_arrays(targets[ii], parvars=parvars)
    else:
        yy_cl, xx_fobs, params_cl = detstats.get_anis_arrays(targets[ii], parvars=parvars)


    colors = cmaps[ii](np.linspace(0,1,len(parvars)))
    ll=1
    handles = []
    for vv, var in enumerate(parvars):
        yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
        pp = 68
        med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        hh, = ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv], zorder=6-vv)
        handles.append(hh)
        ax.fill_between(xx, *conf, alpha=0.25, color=colors[vv], zorder=3-vv)

    ax.legend(handles=handles, loc='lower right', ncols=3, title = plot.PARAM_KEYS[targets[ii]],
              labelspacing=0.1, borderpad=0.1, columnspacing=0.3, handletextpad=0.1, frameon=False, alignment='right')
    # ax.text(0.01,0.99, plot.PARAM_KEYS[targets[ii]], transform = ax.transAxes, va='top', ha='left')
fig.subplots_adjust(wspace=0, hspace=0)

savename = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/anisotropy'
savename += "/C1C0_l10_pv3.png"
fig.savefig(savename, dpi=200, bbox_inches='tight')

# Middle Only

In [ ]:
BUILD_ARRAYS=False
target='gsmf_phi0'
parvars=[0,10,20]
lmax=8

xlabel = plot.LABEL_GW_FREQUENCY_YR
ylabel = r"$C_\ell/C_0$"

# colors = ['#0a5da4', '#00ba44', '#ff9503', '#fe2c02', '#845b98', '#474747']
colors = cm.plasma(np.linspace(0,1,lmax))

fig, ax = plot.figax_single(ncols=1, nrows=1, 
                             sharey=True, sharex=True)
fig.text(0.55, 0.08, xlabel, va='center', ha='center')
fig.text(0.05, 0.55, ylabel, va='center', ha='center', rotation='vertical')
ax.set_title("nloudest=10", fontsize=12)


if BUILD_ARRAYS:
    yy_cl, xx_fobs, params_cl = detstats.build_anis_arrays(target, parvars=parvars)
else:
    yy_cl, xx_fobs, params_cl = detstats.get_anis_arrays(target, parvars=parvars)

yy_cl = yy_cl[int(len(parvars)/2)]
xx = xx_fobs*YR

handles = []
for ll in range(1,lmax+1):  
    yy = yy_cl[:,:,ll]/yy_cl[:,:,0]
    hh, = ax.plot(xx, np.median(yy, axis=-1), label=f"{ll}", alpha=0.75, color=colors[ll-1])
    handles.append(hh)
    cc = hh.get_color()
    for pp in [50,95]:
        lo, hi = np.percentile(yy, (50-pp/2, 50+pp/2), axis=-1)
        ax.fill_between(xx, lo, hi, color=colors[ll-1], alpha=0.2, )
    
ax.legend(handles=handles)
# ax.legend(handles=handles, loc='lower right', ncols=3, title = plot.PARAM_KEYS[targets[ii]],
#               labelspacing=0.1, borderpad=0.1, columnspacing=0.3, handletextpad=0.1, frameon=False, alignment='right')
    # ax.text(0.01,0.99, plot.PARAM_KEYS[targets[ii]], transform = ax.transAxes, va='top', ha='left')
# fig.subplots_adjust(wspace=0, hspace=0)


# 100 Single Sources

In [ ]:
BUILD_ARRAYS=False



xlabel = plot.LABEL_GW_FREQUENCY_YR
ylabel = r"$C_1/C_0$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
cmaps = [
    cmap_Greens, cmap_Greens,
    cmap_Oranges, cmap_Oranges,
    cmap_Blues, cmap_Blues
]
parvars = [0,1,2]



fig, ax = plot.figax_single()
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)
ax.set_title("nloudest=100", fontsize=12)

ii = 0

if BUILD_ARRAYS:
    yy_cl, xx_fobs, params_cl = detstats.build_anis_arrays(targets[ii], 
        nvars=3, nreals=100, nloudest=100, parvars=parvars)
else:
    yy_cl, xx_fobs, params_cl = detstats.get_anis_arrays(targets[ii], 
        nvars=3, nreals=100, nloudest=100, parvars=parvars)

colors = cmaps[ii](np.linspace(0,1,len(parvars)))
handles = draw_anis_vars(ax, targets[ii], xx_fobs*YR, yy_cl, params_cl, colors, parvars)
ax.legend(handles=handles, loc='lower right', ncols=3, title = plot.PARAM_KEYS[targets[ii]],
            labelspacing=0.1, borderpad=0.1, columnspacing=0.3, handletextpad=0.1, frameon=False, alignment='right')
# ax.text(0.01,0.99, plot.PARAM_KEYS[targets[ii]], transform = ax.transAxes, va='top', ha='left')



In [ ]:
BUILD_ARRAYS=False
target='gsmf_phi0'
parvars=[0,1,2]
vv = int(len(parvars)/2)
lmax=8

xlabel = plot.LABEL_GW_FREQUENCY_YR
ylabel = r"$C_\ell/C_0$"
# colors = ['#0a5da4', '#00ba44', '#ff9503', '#fe2c02', '#845b98', '#474747']
colors = cm.plasma(np.linspace(0,1,lmax))

fig, ax = plot.figax_single(ncols=1, nrows=1, 
                             sharey=True, sharex=True)
fig.text(0.55, 0.08, xlabel, va='center', ha='center')
fig.text(0.05, 0.55, ylabel, va='center', ha='center', rotation='vertical')
ax.set_title("nloudest=100", fontsize=12)


if BUILD_ARRAYS:
    yy_cl, xx_fobs, params_cl = detstats.build_anis_arrays(targets[ii], 
        nvars=3, nreals=100, nloudest=100, parvars=parvars)
else:
    yy_cl, xx_fobs, params_cl = detstats.get_anis_arrays(targets[ii], 
        nvars=3, nreals=100, nloudest=100, parvars=parvars)

handles = []
for ll in range(1,lmax+1):  
    yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0]
    hh, = ax.plot(xx, np.median(yy, axis=-1), label=f"{ll}", alpha=0.75, color=colors[ll-1])
    handles.append(hh)
    cc = hh.get_color()
    for pp in [50,95]:
        lo, hi = np.percentile(yy, (50-pp/2, 50+pp/2), axis=-1)
        ax.fill_between(xx, lo, hi, color=colors[ll-1], alpha=0.2, )
    
ax.legend(handles=handles)
# ax.legend(handles=handles, loc='lower right', ncols=3, title = plot.PARAM_KEYS[targets[ii]],
#               labelspacing=0.1, borderpad=0.1, columnspacing=0.3, handletextpad=0.1, frameon=False, alignment='right')
    # ax.text(0.01,0.99, plot.PARAM_KEYS[targets[ii]], transform = ax.transAxes, va='top', ha='left')
# fig.subplots_adjust(wspace=0, hspace=0)


In [ ]:
BUILD_ARRAYS=False

xlabel = plot.LABEL_GW_FREQUENCY_YR
ylabel = r"$C_1/C_0$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
cmaps = [
    cmap_Greens, cmap_Greens,
    cmap_Oranges, cmap_Oranges,
    cmap_Blues, cmap_Blues
]
parvars = [0,1,2]


fig, ax = plot.figax_single()
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)
ax.set_title("nloudest=100, nside=32", fontsize=12)

ii = 0

if BUILD_ARRAYS:
    yy_cl, xx_fobs, params_cl = detstats.build_anis_arrays(targets[ii], 
        nvars=3, nreals=100, nloudest=100, parvars=parvars, nside=32)
else:
    yy_cl, xx_fobs, params_cl = detstats.get_anis_arrays(targets[ii], 
        nvars=3, nreals=100, nloudest=100, parvars=parvars, nside=32)

colors = cmaps[ii](np.linspace(0,1,len(parvars)))
handles = draw_anis_vars(ax, targets[ii], xx_fobs*YR, yy_cl, params_cl, colors, parvars)
ax.legend(handles=handles, loc='lower right', ncols=3, title = plot.PARAM_KEYS[targets[ii]],
            labelspacing=0.1, borderpad=0.1, columnspacing=0.3, handletextpad=0.1, frameon=False, alignment='right')
# ax.text(0.01,0.99, plot.PARAM_KEYS[targets[ii]], transform = ax.transAxes, va='top', ha='left')



# 10 Loudest Again

In [ ]:
BUILD_ARRAYS=False

xlabel = plot.LABEL_GW_FREQUENCY_YR
ylabel = r"$C_1/C_0$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
cmaps = [
    cmap_Greens, cmap_Greens,
    cmap_Oranges, cmap_Oranges,
    cmap_Blues, cmap_Blues
]
parvars = [0,10,20]



fig, ax = plot.figax_single()
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)
ax.set_title("nloudest=10", fontsize=12)

ii = 0

if BUILD_ARRAYS:
    yy_cl, xx_fobs, params_cl = detstats.build_anis_arrays(targets[ii], 
        nvars=NVARS, nreals=NREALS, nloudest=NLOUDEST, parvars=parvars)
else:
    yy_cl, xx_fobs, params_cl = detstats.get_anis_arrays(targets[ii], 
        nvars=NVARS, nreals=NREALS, nloudest=NLOUDEST, parvars=parvars)

colors = cmaps[ii](np.linspace(0,1,len(parvars)))
handles = draw_anis_vars(ax, targets[ii], xx_fobs*YR, yy_cl, params_cl, colors, parvars)
ax.legend(handles=handles, loc='lower right', ncols=3, title = plot.PARAM_KEYS[targets[ii]],
            labelspacing=0.1, borderpad=0.1, columnspacing=0.3, handletextpad=0.1, frameon=False, alignment='right')
# ax.text(0.01,0.99, plot.PARAM_KEYS[targets[ii]], transform = ax.transAxes, va='top', ha='left')



In [ ]:
BUILD_ARRAYS=True

xlabel = plot.LABEL_GW_FREQUENCY_YR
ylabel = r"$C_1/C_0$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
cmaps = [
    cmap_Greens, cmap_Greens,
    cmap_Oranges, cmap_Oranges,
    cmap_Blues, cmap_Blues
]
parvars = [0,10,20]


fig, ax = plot.figax_single()
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)
ax.set_title("nloudest=10", fontsize=12)

ii = 0

if BUILD_ARRAYS:
    yy_cl, xx_fobs, params_cl = detstats.build_anis_arrays(targets[ii], 
        nvars=NVARS, nreals=NREALS, nloudest=NLOUDEST, parvars=parvars, nside=32)
else:
    yy_cl, xx_fobs, params_cl = detstats.get_anis_arrays(targets[ii], 
        nvars=NVARS, nreals=NREALS, nloudest=NLOUDEST, parvars=parvars, nside=32)

colors = cmaps[ii](np.linspace(0,1,len(parvars)))
handles = draw_anis_vars(ax, targets[ii], xx_fobs*YR, yy_cl, params_cl, colors, parvars)
ax.legend(handles=handles, loc='lower right', ncols=3, title = plot.PARAM_KEYS[targets[ii]],
            labelspacing=0.1, borderpad=0.1, columnspacing=0.3, handletextpad=0.1, frameon=False, alignment='right')
# ax.text(0.01,0.99, plot.PARAM_KEYS[targets[ii]], transform = ax.transAxes, va='top', ha='left')

